## 다중분류

책에 쓰여있는대로, 여러가지 클래스를 구분하기 위해서 랜덤 포레스트나 나이브 베이즈, KNN 같이 애초에 여러개의 class를 구분할 수 있는 분류기를 쓸 수도 있지만, 이진 분류기를 반복연산하여 분류할 수도 있습니다. 

이 경우 OvO / OvR을 전략으로 채택할 수 있습니다. 책에도 써 있듯이, 전자는 각각의 조합마다 이진 분류기를 훈련시키는 방법을 말합니다. 0-1, 0-2 ... 9-7, 9-8 이런 식으로 각각의 분류기를 통과하고 나서, 가장 많이 채택된 값을 선택하는 방식으로 작동합니다. 

후자는 특정 숫자 하나만 구별하는 이진 분류기를 여러개 붙이는 방식으로 만들 수 있습니다. 가장 결정 점수가 높은 값을 반환하겠지요. 앞서 3.3에 등장하는 Never5classifier 10개를 붙여논 버전으로 이해하면 될 것 같습니다. OvO부터 살펴보겠습니다. 기본적으로 우리가 사용했던 mnist 데이터를 사용합니다. 

In [1]:
from sklearn.datasets import fetch_openml
import numpy as np
mnist = fetch_openml('mnist_784', version=1)

sklearn 버전이 올라가면서, 책에 있는 fetch_mldata를 더 이상 지원하지 않게 되었습니다. 이제 mnist데이터를 가져오기 위해서 fetch_openml을 사용합니다. 

In [2]:
X, y = mnist["data"], mnist["target"]
y = y.astype(np.int8)
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]
shuffle_index = np.random.permutation(60000)
X_train, y_train = X_train[shuffle_index], y_train[shuffle_index]

그런데 이렇게 가저오는 mnist data의 경우 y의 변수형이 조금 달라서 그냥 넣으면 분석이 안됩니다. astype에 넣어서 넘파이 데이터로 변환 후에 사용합니다. 아무튼 test데이터와 train데이터를 분류했습니다. 이제 OvO, OvR 를 생성해 봅시다.

## OvO

In [3]:
from sklearn.linear_model import SGDClassifier # SGDClassifier : 확률적 경사하강법을 사용하겠다는 선언.
from sklearn.multiclass import OneVsOneClassifier 

기본적인 이진 분류기를 위해서 SGDClassifier를 불러왔습니다. sklearn.multiclass는 다중분류를 위한 하부 패키지입니다. 그 밑에는OneVsRestClassifier와 OneVsOneClassifier, OutputCodeClassifier가 있습니다. 마지막은 Class들을 0~1의 유클리드 공간에 사영한 뒤에, 그걸 학습하고, 예측은 학습한 것을 기반으로 어떻게 한다는데, 열심히 읽어보았지만, 도저히 이해가 안됩니다.

일단 OvO부터 봅시다.

In [4]:
ovo_clf = OneVsOneClassifier(SGDClassifier(random_state = 2014150099))
ovo_clf.fit(X_train, y_train)

OneVsOneClassifier(estimator=SGDClassifier(alpha=0.0001, average=False,
                                           class_weight=None,
                                           early_stopping=False, epsilon=0.1,
                                           eta0=0.0, fit_intercept=True,
                                           l1_ratio=0.15,
                                           learning_rate='optimal',
                                           loss='hinge', max_iter=1000,
                                           n_iter_no_change=5, n_jobs=None,
                                           penalty='l2', power_t=0.5,
                                           random_state=2014150099,
                                           shuffle=True, tol=0.001,
                                           validation_fraction=0.1, verbose=0,
                                           warm_start=False),
                   n_jobs=None)

#### Parameter : OneVsOneClassifier(estimator, n_jobs=None)

기본적으로 갖는 parameter가 2개 밖에 없습니다. 앞에는 OvO방식으로 작동할 estimator를 넣어줍니다. 당연히 fit 메서드를 가지고 있어야 하며, 결과를 도출할 decision_function 메서드나 샘플의 클래스별 확률을 도출하는 predict_proba 중 하나를 가져야 합니다. 

n_jobs를 통해 프로세서의 코어 숫자(작업 개수)를 지정할 수 있습니다. 기본은 1이고, 어차피 CPU가 여러개가 아니면 늘려봐야 의미는 없습니다. -1을 지정하면 모든 작업을 총 동원하여 학습합니다. 

In [5]:
print(ovo_clf.predict([X[36000]]))
print(ovo_clf.decision_function(X_train[:10]))
print(ovo_clf.predict(X_train[:10]))

[9]
[[ 1.66666719  5.33333144  0.66666789  7.33333253  4.33333187  2.66666791
  -0.33333322  9.33333315  6.33332537  8.3333331 ]
 [ 1.66666719  7.33333296  3.66669361  8.33333293  3.66666749  5.33333267
   0.66666687  0.66666689  9.33333315  4.33333156]
 [-0.33333314  3.33333101  6.33333244  7.33333288  5.3333321   5.33333229
   0.66666683  1.66666713  9.33333316  7.33333288]
 [ 2.66666685 -0.33333326  3.66666731  0.6666679   9.33333322  1.66666803
   5.33333282  6.33333312  7.33333283  8.33333314]
 [ 9.33333323  0.66666681  6.33333281  5.33333245 -0.3333332   7.333333
   7.33333297  1.66666683  4.33333309  3.66666731]
 [ 8.33333292  1.66666684  4.33333078  3.66666747  6.33333259  2.66666721
   9.33333326 -0.3333332   7.33333243  1.66666705]
 [ 1.6666669   1.66666776  2.66666847  5.33333249  8.33333303  2.66666714
  -0.33333317  7.33333306  6.33333167  9.33333315]
 [ 2.66666683  0.6666668   1.66666695  9.33333323  4.33333284  8.33333313
  -0.33333328  6.33333313  5.33333315  7.33333319

In [6]:
print(ovo_clf.get_params(deep=True))
print("===============================")
print(ovo_clf.score(X_train, y_train))

{'estimator__alpha': 0.0001, 'estimator__average': False, 'estimator__class_weight': None, 'estimator__early_stopping': False, 'estimator__epsilon': 0.1, 'estimator__eta0': 0.0, 'estimator__fit_intercept': True, 'estimator__l1_ratio': 0.15, 'estimator__learning_rate': 'optimal', 'estimator__loss': 'hinge', 'estimator__max_iter': 1000, 'estimator__n_iter_no_change': 5, 'estimator__n_jobs': None, 'estimator__penalty': 'l2', 'estimator__power_t': 0.5, 'estimator__random_state': 2014150099, 'estimator__shuffle': True, 'estimator__tol': 0.001, 'estimator__validation_fraction': 0.1, 'estimator__verbose': 0, 'estimator__warm_start': False, 'estimator': SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=2014150099, shuffle=T

#### Methods :

##### decision_fucntion(self, X)
    OvO에서 X에 대해 사용한 decision_funciton을 반환합니다.

##### fit(self,X,y)
X,y에 대해 모델을 피팅합니다. 

##### partial_fit(self,X,y[,classes])
지정해준 y class에 대해서만 피팅합니다 

##### get_params(self, deep = True)
파라미터들을 반환합니다. deep=True를 지정해주면 subobjects를 생성합니다.

##### predict(self, X)
피팅한 모델을 기반으로 X에 대해 예측합니다.

##### score(self, X,y, sample_weight=None)
mean accuracy를 반환합니다. sample_weight를 통해 가중치에 해당하는 array를 지정해줄 수도 있습니다.

In [8]:
print(len(ovo_clf.estimators_))
print((ovo_clf.classes_))

45
[0 1 2 3 4 5 6 7 8 9]


#### Attributes :
##### estimators_
n(n-1)/2개의 estimator들의 리스트입니다. 

##### classes_
class의 label을 포함하는 array입니다. 

##### pairwise_indices_
estimator가 _pairwise attribute를 가진다면 estimator를 훈련하는데 썼던 sample의 indicies들의 리스트를 출력합니다.

## OvR

In [9]:
from sklearn.multiclass import OneVsRestClassifier
ovr_clf = OneVsRestClassifier(SGDClassifier(random_state = 2014150099))

In [10]:
ovr_clf.fit(X_train[:2000], y_train[:2000])

OneVsRestClassifier(estimator=SGDClassifier(alpha=0.0001, average=False,
                                            class_weight=None,
                                            early_stopping=False, epsilon=0.1,
                                            eta0=0.0, fit_intercept=True,
                                            l1_ratio=0.15,
                                            learning_rate='optimal',
                                            loss='hinge', max_iter=1000,
                                            n_iter_no_change=5, n_jobs=None,
                                            penalty='l2', power_t=0.5,
                                            random_state=2014150099,
                                            shuffle=True, tol=0.001,
                                            validation_fraction=0.1, verbose=0,
                                            warm_start=False),
                    n_jobs=None)

모든 옵션이 OvO와 동일합니다. 간단하게 값들만 보고 가도록 하겠습니다. 

In [15]:
print(ovr_clf.predict([X[36000]]))
print(ovr_clf.decision_function(X_train[:10]))
print(ovr_clf.predict(X_train[:10]))
print("----------------------------")
print(ovo_clf.predict(X_train[:10]))

[4]
[[-2430660.38570224 -1374895.82472558  -343654.43203806 -1271980.74408565
  -1210287.78091764  -991202.9029372  -2228723.70687807   853247.45832273
   -839769.94370653  -255615.04170723]
 [-2883222.75909643 -1862094.04907251  -772102.86970634  -918711.30577438
  -2078019.49094479  -664581.64811572 -3116814.45375546 -2505855.99639001
    305056.40265691  -879682.18294531]
 [-2842158.62924695 -1530468.64357482  -723284.27249715 -1127564.95779074
  -2353566.24565518  -844430.60719311 -3714747.35440058 -1653014.75850683
    822352.77677347  -897229.34996136]
 [-4155165.11659277 -5945917.31305457 -2849206.36360626 -1702408.07504932
   1558529.34316371 -2111018.8745732  -2331077.47852065 -1955388.79016059
   -439404.88342495 -1866716.16097477]
 [ 2461695.57214283 -4464676.55108476 -1136874.78927433  -593698.28584348
  -4426942.96592059 -1099420.93174278 -1647302.698623   -2852427.68401251
   -981109.50931963 -1793986.73334938]
 [-3779340.60827205 -3158495.42471582 -1994837.21946557 -2792

In [12]:
print(ovr_clf.get_params(deep=True))
print("===============================")
print(ovr_clf.score(X_train, y_train))

{'estimator__alpha': 0.0001, 'estimator__average': False, 'estimator__class_weight': None, 'estimator__early_stopping': False, 'estimator__epsilon': 0.1, 'estimator__eta0': 0.0, 'estimator__fit_intercept': True, 'estimator__l1_ratio': 0.15, 'estimator__learning_rate': 'optimal', 'estimator__loss': 'hinge', 'estimator__max_iter': 1000, 'estimator__n_iter_no_change': 5, 'estimator__n_jobs': None, 'estimator__penalty': 'l2', 'estimator__power_t': 0.5, 'estimator__random_state': 2014150099, 'estimator__shuffle': True, 'estimator__tol': 0.001, 'estimator__validation_fraction': 0.1, 'estimator__verbose': 0, 'estimator__warm_start': False, 'estimator': SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=2014150099, shuffle=T

In [16]:
print(len(ovr_clf.estimators_))
print(len(ovo_clf.estimators_))

10
45


이처럼 각각이 연산한 estimator의 갯수가 다른 것을 확인할 수 있습니다.

## Random Forest Classifier

자체적으로 다중분류를 수행하는 RandomForest가 소개되어있어서 조사했습니다. sklearn.ensemble에서 임포트합니다. 

In [25]:
from sklearn.ensemble import RandomForestClassifier
forest_clf = RandomForestClassifier(random_state = 2014150099)
forest_clf.fit(X_train, y_train)

C:\Users\USER\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=2014150099,
                       verbose=0, warm_start=False)

#### Parameter : (n_estimators=’warn’, criterion=’gini’, max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=’auto’, max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, bootstrap=True, oob_score=False, n_jobs=None, random_state=None, verbose=0, warm_start=False, class_weight=None)

뭐가 엄청 많습니다. 하지만 대부분 tree에 대한 옵션이고, 그마저도 죄다 optional이라 ()만 해도 알아서 디폴트로 잘 굴려줍니다.

##### n_estimators = int
트리의 개수를 정합니다. 디폴트는 10이나 0.22부터 100으로 증가됩니다. 

##### criterion = 'gini / entropy'
평가 기준 함수를 정합니다. 둘 중하나를 선택할 수 있으며 지니가 디폴트입니다.

##### min / max들
tree에 대해서 여러가지 제한을 정합니다. 이 부분에 대한 설명은 
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html
여길 참조하세요. 

##### bootstrap = True / False
부트스트랩 데이터를 사용할지 여부를 정합니다. 기본으로 True며, False를 설정하면 그냥 전체 데이터를 사용합니다. 

##### random_state / n_jobs 
상동

##### class_weight 
가중치를 정합니다. 디폴트는 없음이며, dictionary형태로 직접 입력하거나 balanced / balanced_subsample를 통해 자동으로 지정해줄 수 있습니다. 둘의 차이는 부트스트랩 대이터를 쓰지않거나, 쓰거나의 차이입니다. 

In [28]:
forest_clf.predict(X_train[:10])

array([7, 8, 8, 4, 0, 6, 9, 3, 8, 1], dtype=int8)

In [ ]:
forest_clf.predict_proba(X_train[:10])
forest_clf.score(X_train,y)

#### Methods :
##### decision_function(self, X),  fit(self,X,y), get_params(self, deep), partial_fit(self,X,y[,classes]), predict(self,X),  score(self, X,y[sample_weight]
상동

##### predict_proba(self, X)
class에 대한 확률추정치를 반환합니다.

#### Attributes:
##### estimators_, classes_
상동

##### label_binarizer_
binary를 multi로, multi를 binary로 변환하는데 쓰였던 object 반환

##### multilabel_
multi인지 여부를 논리값으로 가집니다.

## 번외1) KNN

그 외에도 여러가지 Multilabel classifier가 존재합니다. 책 뒤에서 다루는 SVM도 그 중 하나입니다. 이는 뒤에서 볼 거니까 위해 남기고 KNN과 Naive-Bayesian 두 가지만 더 보겠습니다.

In [20]:
from sklearn.neighbors import KNeighborsClassifier
knn=KNeighborsClassifier() 
knn.fit(X_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [22]:
print(knn.predict(X_train[:10]))
print(ovo_clf.predict(X_train[:10]))

[7 8 8 4 0 6 9 3 8 1]
[7 8 8 4 0 6 9 3 8 1]


x_KNN은 예측하고 싶은 값의 주변 K개의 중에 가장 빈번한 값을 반환하는 단순한 알고리즘입니다. 하지만 생각보다 결과가 괜찮습니다. 위의 코드에서 ovo_clf와의 결과가 동일한 것을 확인할 수 있습니다. 

보다 구체적으로 KNN과 ovo_clf, ovr_clf, forest_clf 4가지를 비교해보겠습니다. x_test 앞의 1000개의 값을 예측시켜보고, y_test를 얼마나 잘 맞추는지 한 번 보죠.

In [27]:
y_test_pred_ovo = ovo_clf.predict(X_test[:1000])
y_test_pred_ovr = ovr_clf.predict(X_test[:1000])
y_test_pred_forest = forest_clf.predict(X_test[:1000])
y_test_pred_knn = knn.predict(X_test[:1000])

print('Misclassified test samples of OvO : %d' %(y_test[:1000]!=y_test_pred_ovo).sum())
print('Misclassified test samples of OvR : %d' %(y_test[:1000]!=y_test_pred_ovr).sum())
print('Misclassified test samples of Forest : %d' %(y_test[:1000]!=y_test_pred_forest).sum())
print('Misclassified test samples of KNN : %d' %(y_test[:1000]!=y_test_pred_knn).sum())

Misclassified test samples of OvO : 91
Misclassified test samples of OvR : 176
Misclassified test samples of Forest : 65
Misclassified test samples of KNN : 39


와우. 심지어 RandomForest보다도  결과가 좋군요. KNN은 이처럼 생각보다 좋은 결과를 반환하는 경우가 잦습니다. 이제 Parameter와 method들을 살펴봅시다. 

#### Parameters: 
##### n_neighbors
K의 숫자를 정합니다. 5를 기본값으로 가집니다.  
##### weigth 
가중치를 정합니다. Uniform이 기본값이며 ‘distance’로 설정하면 거리의 역수로 가중치를 설정합니다. 즉 가까울수록 더 큰 가중치를 받습니다. 
##### metric 
거리 계산을 하는 방식을 지정합니다. 기본은 ‘minkowski’로 설정되어있습니다. minkowski 방식은 밑에 나오는 P와 함께 쓰입니다. 
##### P
Minkowski의power parameter를 정합니다. 1이면 멘하탄 거리, 2이면 유클리드 거리로 계산합니다.

#### Methods
predict(self, X) , predict_proba(self, X), Score(self, X,y[,sample_weight]) 등을 갖습니다. 반복되니 생략합니다. 

## 번외2) Naive Bayes

나이브 베이즈는 조건부 확률 기반의 분류 방식입니다. 분류를 원하는 특성 변수에 대해 각각의 class에 대한 조건부 확률을 구하고, 그것들 간의 비를 통해 어떤 class일 확률이 더 높은지를 결정, 반환합니다. 이건 제가 이해하긴 했는데 수식 없이 글로는 설명이 힘드네요...톡방에 자료를 따로 공유하겠습니다. 

코드 보시죠.

In [33]:
from sklearn.naive_bayes import MultinomialNB 
mnb=MultinomialNB()
mnb.fit(X_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

다변수 나이브 베이즈는 naive_bayes에서 import합니다. fit, partial_fit, predict와 predict_proba, predict_log_proba, score등을 메서드로 가집니다. 위에서 다 본 것들이니 생략합니다. 

In [34]:
print(mnb.predict(X_train[:10]))
print(ovo_clf.predict(X_train[:10]))

y_test_pred_mnb = mnb.predict(X_test[:1000])
print('Misclassified test samples of MNB : %d' %(y_test[:1000]!=y_test_pred_mnb).sum())

[7 8 8 4 0 6 9 3 8 1]
[7 8 8 4 0 6 9 3 8 1]
Misclassified test samples of MNB : 189


생각보다 결과가 좋지는 않네요. Naive Bayes는 독립성과 조건부 x의 정규성을 임의로 가정하는 모델이기 때문에, 이 가정에서 벗어나는 데이터일수록 좋지 못한 결과를 반환합니다. parameter를 보고 마무리하겠습니다. 

#### Parameters:
##### alpha 
smoothing을 얼마나 할지를 지정합니다. 커질수록 smooth해집니다. 1이 기본입니다. 

##### fit_prior
prior prop을 사용하지 여부를 boolean으로 입력합니다. 만일 아니라면 그냥 다 동등하게 간주하여 class를 학습합니다. 기본은 True입니다. 

##### class_prior 
class들의 Prior probabilities를 넘파이 어레이 형태로 입력합니다. 입력하지 않으면 데이터 기반으로 추정하지만, 이걸 지정해주면 자의로 정할 수도 있습니다.